In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
path = '/content/drive/MyDrive/praktikum_ml/praktikum03'

In [ ]:
import pandas as pd
# buat dataframe read data
df = pd.read_csv(path + '/Data/socr.csv',sep=',')
df

,Index,Height(Inches),Weight(Pounds)
0,1,65.78331,112.9925
1,2,71.51521,136.4873
2,3,69.39874,153.0269
3,4,68.21660,142.3354
4,5,67.78781,144.2971
...,...,...,...
24995,24996,69.50215,118.0312
24996,24997,64.54826,120.1932
24997,24998,64.69855,118.2655
24998,24999,67.52918,132.2682


In [ ]:
df.describe()

,Index,Height(Inches),Weight(Pounds)
count,25000.000000,25000.000000,25000.000000
mean,12500.500000,67.993114,127.079421
std,7217.022701,1.901679,11.660898
min,1.000000,60.278360,78.014760
25%,6250.750000,66.704397,119.308675
50%,12500.500000,67.995700,127.157750
75%,18750.250000,69.272958,134.892850
max,25000.000000,75.152800,170.924000


In [ ]:
##konversi data menjadi satuan CM dan KG
df['Height(cm)'] = df['Height(Inches)'] * 2.54
df['Weight(kg)'] = df['Weight(Pounds)'] * 0.45
df

,Index,Height(Inches),Weight(Pounds),Height(cm),Weight(kg)
0,1,65.78331,112.9925,167.089607,50.846625
1,2,71.51521,136.4873,181.648633,61.419285
2,3,69.39874,153.0269,176.272800,68.862105
3,4,68.21660,142.3354,173.270164,64.050930
4,5,67.78781,144.2971,172.181037,64.933695
...,...,...,...,...,...
24995,24996,69.50215,118.0312,176.535461,53.114040
24996,24997,64.54826,120.1932,163.952580,54.086940
24997,24998,64.69855,118.2655,164.334317,53.219475
24998,24999,67.52918,132.2682,171.524117,59.520690


In [ ]:
df1 = (
    df[["Height(Inches)", "Weight(Pounds)"]]
    .rename(columns={"Height(Inches)": "tinggi_cm", "Weight(Pounds)": "berat_pound"})
    .assign(
        tinggi_cm=lambda d: d["tinggi_cm"] * 2.54,
        berat_kg=lambda d: d["berat_pound"] * 0.45359237
    )
    .round({"tinggi_cm": 2, "berat_kg": 2})
    .copy()
)
df1.head()

,tinggi_cm,berat_pound,berat_kg
0,167.09,112.9925,51.25
1,181.65,136.4873,61.91
2,176.27,153.0269,69.41
3,173.27,142.3354,64.56
4,172.18,144.2971,65.45


In [ ]:
from sklearn.model_selection import train_test_split

# variabel independent/bebas
X = df1[['tinggi_cm']]

# variabel dependent/terikat/target
y = df1[['berat_kg']]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=7)

## cetak data training head
X_train.head()

,tinggi_cm
3253,167.14
18789,178.49
4466,169.75
19269,181.97
6527,177.60


In [ ]:
from sklearn.linear_model import LinearRegression

##buat objek model insan dari class LinearRegression
model = LinearRegression()

##lakukan proses training
model.fit(X_train, y_train)


LinearRegression()

In [ ]:
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score,mean_absolute_error,mean_squared_error

y_pred = model.predict(X_test)
r2 = r2_score(y_test, y_pred)

df_resume = pd.DataFrame()
df_resume ['R2'] = r2
df_resume ['Koefisien (kg/cm)'] = model.coef_[0]
df_resume ['Intersep (kg)'] = model.intercept_
df_resume ['MAE (kg)'] = model.intercept_
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)


df_resume['MSE (kg)'] = mse
df_resume['RMSE (kg)'] = rmse
df_resume

,R2,Koefisien (kg/cm),Intersep (kg),MAE (kg),MSE (kg),RMSE (kg)
0,NaN,0.551823,-37.657089,-37.657089,21.242938,4.609006


In [ ]:
slope = float(model.coef_[0])
intercept = float(model.intercept_)

print(f"Persamaan : y = {slope:.3f}x + {intercept:.3f}")

Persamaan : y = 0.552x + -37.657


/tmp/ipython-input-4179704099.py:1: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  slope = float(model.coef_[0])
/tmp/ipython-input-4179704099.py:2: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  intercept = float(model.intercept_)


In [ ]:
y_pred_test = model.predict(X_test)

# Buat tabel hasil (tinggi, aktual, prediksi, dan error)
hasil = pd.DataFrame({
    "Tinggi (cm)": X_test["tinggi_cm"].to_numpy().ravel(),
    "Berat Aktual (kg)": y_test.to_numpy().ravel(),
    "Berat Prediksi (kg)": y_pred_test.ravel()
})

# Hitung selisih error
hasil["Selisih error (kg)"] = hasil["Berat Prediksi (kg)"] - hasil["Berat Aktual (kg)"]

# Akurasi per-baris (%)
denom = hasil["Berat Aktual (kg)"].replace(0, np.nan)
hasil["Akurasi (%)"] = (1 - (hasil["Selisih error (kg)"].abs() / denom)).clip(lower=0, upper=1) * 100

# Tampilkan hasil
hasil



,Tinggi (cm),Berat Aktual (kg),Berat Prediksi (kg),Selisih error (kg),Akurasi (%)
0,174.73,50.16,58.762990,8.602990,82.848904
1,171.31,50.33,56.875754,6.545754,86.994329
2,169.29,58.22,55.761071,-2.458929,95.776488
3,163.30,58.92,52.455650,-6.464350,89.028598
4,170.52,63.06,56.439814,-6.620186,89.501766
...,...,...,...,...,...
4995,178.75,56.59,60.981319,4.391319,92.240114
4996,163.05,47.45,52.317694,4.867694,89.741425
4997,166.51,52.46,54.227003,1.767003,96.631715
4998,167.70,49.90,54.883672,4.983672,90.012681


In [ ]:
##data testing
Data = pd.read_csv(path + '/Data/berat_TI_2022.csv',sep =',')

y_pred = model.predict(Data[['tinggi_cm']])
Data['prediksi_kg'] = y_pred
Data["selisih error (kg)"] = Data["prediksi_kg"] - Data["actual_berat"]
denom = Data["actual_berat"].replace(0, np.nan)
Data["akurasi (%)"] = (1 - (Data["selisih error (kg)"].abs() / denom)).clip(lower=0, upper=1) * 100
Data

,no,nama,tinggi_cm,actual_berat,prediksi_kg,selisih error (kg),akurasi (%)
0,1,rafa,168,58.5,55.049219,-3.450781,94.101229
1,2,tiiara,162,78.2,51.738280,-26.461720,66.161483
2,3,tia,155,52.7,47.875517,-4.824483,90.845383
3,4,keyla,160,70.3,50.634633,-19.665367,72.026505
4,5,Bani,183,65.8,63.326568,-2.473432,96.240985


In [ ]:
Data.describe()

,no,tinggi_cm,actual_berat,prediksi_kg,selisih error (kg),akurasi (%)
count,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000
mean,3.000000,165.600000,65.100000,53.724843,-11.375157,83.875117
std,1.581139,10.784248,9.955652,5.950999,10.969045,13.786181
min,1.000000,155.000000,52.700000,47.875517,-26.461720,66.161483
25%,2.000000,160.000000,58.500000,50.634633,-19.665367,72.026505
50%,3.000000,162.000000,65.800000,51.738280,-4.824483,90.845383
75%,4.000000,168.000000,70.300000,55.049219,-3.450781,94.101229
max,5.000000,183.000000,78.200000,63.326568,-2.473432,96.240985


In [ ]:
##ambik x testing

X_testing = df_TI_2022[['tinggi_cm']]
X_testing

,tinggi_cm
0,168
1,162
2,155
3,160
4,183
